In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

import warnings 
warnings.filterwarnings("ignore")
import os

Продолжим обработку данных с Твиттера. 

#### 1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
Исключим стоп-слова с помощью stop_words='english'.
Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


In [2]:
#Предобработанные данные взяты с прошлого ДЗ
combine_df = pd.read_pickle('preproc.pkl')
combine_df.head(3)

,id,label,tweet,clean_tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


In [3]:
#For 'tweet_stemmed' 
vectorizer = CountVectorizer(max_df=0.9, stop_words='english', max_features=1000)
stemmed = combine_df['tweet_stemmed'].apply(lambda x: ' '.join(x))
BOW_stemmed = vectorizer.fit_transform(stemmed)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#For 'tweet_lemmatized' 
vectorizer = CountVectorizer(max_df=0.9, stop_words='english', max_features=1000)
lemmatized = combine_df['tweet_lemmatized'].apply(lambda x: ' '.join(x))
BOW_lemmatized = vectorizer.fit_transform(lemmatized)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_lemmatized.toarray(), columns = feature_names)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
Исключим стоп-слова с помощью stop_words='english'.
Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().


In [5]:
#'tweet_stemmed'
vectorizer = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=1000)
BOW_stemmed = vectorizer.fit_transform(stemmed)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.744148,0.392399,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [6]:
# 'tweet_lemmatized'
vectorizer = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=1000)
BOW_lemmatized = vectorizer.fit_transform(lemmatized)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_lemmatized.toarray(), columns = feature_names)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.742113,0.39173,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0


#### 3. Натренируем gensim.models.Word2Vec модель на наших данных.
Тренировать будем на токенизированных твитах combine_df['tweet_token']
Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.


In [13]:
X_train = combine_df['tweet_token'].tolist()

model = Word2Vec(
    X_train, 
    size=200, 
    window=5, 
    min_count=1, 
    sg = 1, 
    hs = 0, 
    negative = 10, 
    workers= 32, 
    seed = 34)

total_examples = len(X_train)
model.train([["hello", "world"]], total_examples=total_examples, epochs=20)

(40, 40)

In [19]:
# min_count = 2 заменила на min_count = 1, т.к. при обучении модель выдает ошибку. 
#Избежать ошибу получилось с такими параметрами.

#### 4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [14]:
model.most_similar(positive = "dinner")

[('wine', 0.9574465751647949),
 ('brunch', 0.9470599889755249),
 ('familytime', 0.9465732574462891),
 ('bbq', 0.9465308785438538),
 ('lunch', 0.9465113878250122),
 ('drinks', 0.9433295726776123),
 ('sushi', 0.9430092573165894),
 ('cocktails', 0.9387565851211548),
 ('funday', 0.9384403824806213),
 ('spa', 0.9370788335800171)]

In [15]:
model.most_similar(positive = "trump")

[('republican', 0.9140926003456116),
 ('carl', 0.8854789733886719),
 ('fuhered', 0.8850727081298828),
 ('campaign', 0.8849924802780151),
 ('ally', 0.8775618672370911),
 ('protesting', 0.8774281144142151),
 ('donald', 0.8756227493286133),
 ('suppoer', 0.8747391700744629),
 ('paladino', 0.8682032823562622),
 ('clinton', 0.8618806600570679)]

#### 5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".


In [16]:
model.wv['food']

array([-0.07993524,  0.59261453,  0.30628115,  0.22630157, -0.00863434,
       -0.23603415, -0.2873638 , -0.18331365,  0.1179215 ,  0.23698375,
       -0.30314764, -0.22683647,  0.12273771, -0.40088162, -0.03026888,
       -0.315941  , -0.01073923,  0.3477114 ,  0.07820822, -0.4973732 ,
       -0.07378529, -0.283644  , -0.15393783,  0.16719164, -0.01926101,
        0.48111248, -0.26565552,  0.39285868,  0.4410533 ,  0.3509875 ,
       -0.16991717, -0.28775576, -0.0336391 ,  0.15547854,  0.17853662,
        0.04625831,  0.06510436,  0.06000191,  0.37798262,  0.18055683,
        0.2872612 , -0.0152347 ,  0.06067169,  0.1492864 , -0.52414185,
        0.01942236, -0.26252607, -0.59346354, -0.5878183 , -0.2296067 ,
        0.13694663, -0.03673977, -0.32178986,  0.28079516,  0.08791833,
       -0.02205667,  0.23969926, -0.12917802, -0.02717474,  0.01532773,
        0.13249822,  0.28433308,  0.34028342,  0.26996624, -0.17355272,
       -0.2938896 , -0.3781974 ,  0.23664069,  0.11658168,  0.10

#### 6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).


In [17]:
wordvec_df = np.zeros((total_examples, 200))
for i, tweet in enumerate(combine_df['tweet_token']):
    vec = np.zeros((1, 200))
    for word in tweet:
        vec += model.wv[word].reshape((1, 200))
    vec /= len(tweet)
    wordvec_df[i] = vec

In [20]:
wordvec_df.shape

(49159, 200)